In [1]:
import nibabel as nib
import numpy as np
from scipy import ndimage

In [4]:
import nibabel as nib
import numpy as np
from scipy import ndimage

def expand_nii(input_file, output_file, margin_pixels=5, fill_holes=True):
    """
    Create a cover (hull) of a binary 3D volume with specified margin and fill interior holes.
    
    Parameters:
    -----------
    input_file : str
        Path to input .nii.gz file containing binary 3D volume
    output_file : str
        Path to save the resulting covered volume
    margin_pixels : int, optional
        Number of pixels for the margin (default: 5)
    fill_holes : bool, optional
        Whether to fill holes inside the object (default: True)
    """
    # Load the binary 3D volume
    img = nib.load(input_file)
    data = img.get_fdata()
    
    # Make sure the data is binary
    binary_data = (data > 0).astype(np.int16)
    
    # Keep a copy of the original binary data for statistics
    original_binary = binary_data.copy()
    
    # Fill holes if requested
    if fill_holes:
        # Fill holes in each 2D slice along all three axes for more complete filling
        filled_data = binary_data.copy()
        
        # Function to fill holes in a 3D volume by processing 2D slices
        def fill_holes_3d(volume):
            result = volume.copy()
            
            # Process along each axis
            for axis in range(3):
                # For each slice along the current axis
                slices = [result.take(i, axis=axis) for i in range(result.shape[axis])]
                
                # Fill holes in each 2D slice
                filled_slices = [ndimage.binary_fill_holes(slice) for slice in slices]
                
                # Put the filled slices back into the volume
                for i, filled_slice in enumerate(filled_slices):
                    # Create the appropriate slice object for the current axis
                    if axis == 0:
                        result[i, :, :] = filled_slice
                    elif axis == 1:
                        result[:, i, :] = filled_slice
                    else:  # axis == 2
                        result[:, :, i] = filled_slice
                        
            return result
        
        # Apply 3D hole filling
        filled_data = fill_holes_3d(binary_data)
        
        # Apply 3D binary closing to ensure complete filling of complex holes
        struct = ndimage.generate_binary_structure(3, 1)
        filled_data = ndimage.binary_closing(filled_data, structure=struct, iterations=3).astype(np.int16)
        
        # Finally, use binary_fill_holes on the entire 3D volume for any remaining holes
        filled_data = ndimage.binary_fill_holes(filled_data).astype(np.int16)
        
        # Use the filled data for further processing
        binary_data = filled_data
    
    # Create a structure element for dilation
    # Using a ball/sphere structure for 3D volumes
    struct_elem = ndimage.generate_binary_structure(3, 1)
    struct_elem = ndimage.iterate_structure(struct_elem, margin_pixels)
    
    # Dilate the binary volume to create the cover with margin
    covered_data = ndimage.binary_dilation(binary_data, structure=struct_elem).astype(np.int16)
    
    # Create a new NIfTI image with the same header
    covered_img = nib.Nifti1Image(covered_data, img.affine, img.header)
    
    # Save the result
    nib.save(covered_img, output_file)
    
    print(f"Cover with {margin_pixels}-pixel margin created and saved to {output_file}")
    
    # Return some statistics
    original_voxels = np.sum(original_binary)
    filled_voxels = np.sum(binary_data)
    covered_voxels = np.sum(covered_data)
    
    holes_filled = filled_voxels - original_voxels
    increase_percentage = ((covered_voxels - original_voxels) / original_voxels) * 100
    
    return {
        "original_volume_voxels": original_voxels,
        "holes_filled_voxels": holes_filled,
        "after_filling_voxels": filled_voxels,
        "final_volume_voxels": covered_voxels,
        "volume_increase_percentage": increase_percentage
    }

In [5]:
input_file = r'C:\Users\acer\Desktop\Project_TMJOA\Data\47-4881 L 2014.nii.gz'  # Replace with your input file path
output_file = r'C:\Users\acer\Desktop\Project_TMJOA\Data\output\47-4881 L 2014_expanded.nii.gz'  # Replace with your desired output file path 
margin = 5

stats = expand_nii(input_file, output_file, margin)

Cover with 5-pixel margin created and saved to C:\Users\acer\Desktop\Project_TMJOA\Data\output\47-4881 L 2014_expanded.nii.gz


In [7]:
import os

input_folder = r"C:\Users\acer\Desktop\Project_TMJOA\Data"
output_folder = r"C:\Users\acer\Desktop\Project_TMJOA\Data\output"
expand_distance = 5

nii_count = len([filename for filename in os.listdir(input_folder) if filename.endswith('.nii.gz')])
print(f"There are {nii_count} .nii.gz files in the {input_folder}")

progress_count = 0

files = sorted(os.listdir(input_folder))

for input_name in files :
    if input_name.endswith('.nii.gz'):
        progress_count += 1
        print(f"[Processing {progress_count} out of {nii_count}]")
        
        input_path = os.path.join(input_folder, input_name)

        output_name = os.path.basename(input_path)
        output_name = os.path.splitext(output_name)[0]
        output_name = os.path.splitext(output_name)[0]
        output_name = f"{output_name}_augmented.nii.gz"

        result = expand_nii(input_path, os.path.join(output_folder, output_name), expand_distance)

There are 3 .nii.gz files in the C:\Users\acer\Desktop\Project_TMJOA\Data
[Processing 1 out of 3]
Cover with 5-pixel margin created and saved to C:\Users\acer\Desktop\Project_TMJOA\Data\output\47-4881 L 2014_augmented.nii.gz
[Processing 2 out of 3]
Cover with 5-pixel margin created and saved to C:\Users\acer\Desktop\Project_TMJOA\Data\output\47-4881 L 2018_augmented.nii.gz
[Processing 3 out of 3]
Cover with 5-pixel margin created and saved to C:\Users\acer\Desktop\Project_TMJOA\Data\output\47-4881 R 2014_augmented.nii.gz


In [ ]:
# Result Example

file_path = r"D:\Kananat\Segmentation\5450 L_segmented.nii.gz"
img = nib.load(file_path)
data = img.get_fdata()
print(data.shape)

slice_number = 260

slice = data[slice_number,:,:]

#expand the edge of the segmentation a little
slice = expand_labels(slice, distance=5)

#fill the hole if there are holes
cv_img = array2cv(slice)
filled_cv_img = fillhole(cv_img)
array_img = cv2array(filled_cv_img)

#display result
f, axarr = plt.subplots(1,2)
axarr[0].set_title("Input")
axarr[0].imshow(data[slice_number,:,:], cmap = 'grey')
axarr[1].set_title("Output")
axarr[1].imshow(array_img, cmap = 'grey')